In [1]:
import os

import re
import pymorphy2
from typing import List

import pandas as pd

In [2]:
# configs
PROJECT_PATH = "/home/alex/paper-2025-anonymous-submission"

In [3]:
dataset = pd.read_json(
    os.path.join(
        PROJECT_PATH,
        "Data/processed_data/dataset.json"
    ),
    orient="index"
)

/home/alex/aspirantura_hse/general_venv/lib/python3.10/site-packages/pandas/core/tools/datetimes.py:557: RuntimeWarning: invalid value encountered in cast
  arr, tz_parsed = tslib.array_with_unit_to_datetime(arg, unit, errors=errors)
/home/alex/aspirantura_hse/general_venv/lib/python3.10/site-packages/pandas/core/tools/datetimes.py:557: RuntimeWarning: invalid value encountered in cast
  arr, tz_parsed = tslib.array_with_unit_to_datetime(arg, unit, errors=errors)


In [4]:
df_gigachat_lite = pd.read_json(
    os.path.join(
        PROJECT_PATH,
        "Data/predictions/wordplay_interpretation_gigachat_lite_predictions.json"
    ),
    orient="index"
)

In [ ]:
df_gigachat_lite.sample(1)

,user_prompt,system_prompt,gigachat_lite_preds
51,Заголовок новости: Смартфоны в танталовых мука...,\nПроанализируй заголовок новости в контексте ...,"В заголовке новости ""Смартфоны в танталовых му..."


In [6]:
df_gigachat_max = pd.read_json(
    os.path.join(
        PROJECT_PATH,
        "Data/predictions/wordplay_interpretation_gigachat_max_predictions.json"
    ),
    orient="index"
)

In [7]:
df_gigachat_max["gigachat_max_pred"].fillna("что-то в вашем вопросе меня смущает.", inplace=True)

In [8]:
df_gigachat_max.sample(1)

,user_prompt,system_prompt,gigachat_max_pred
1025,"Заголовок новости: Главное не Таврида, а участ...",\nПроанализируй заголовок новости в контексте ...,"что-то в вашем вопросе меня смущает. может, по..."


In [9]:
df_yagpt = pd.read_json(
    os.path.join(
        PROJECT_PATH,
        "Data/predictions/wordplay_interpretation_yandex_gpt4_predictions.json"
    ),
    orient="index"
)

In [10]:
df_yagpt.sample(1)

,user_prompt,system_prompt,yagpt_pred
2042,Заголовок новости: И за бот ее бросает. Cодерж...,\nПроанализируй заголовок новости в контексте ...,в заголовке нет игры слов. это перефразированн...


In [11]:
df_mistral = pd.read_json(
    os.path.join(
        PROJECT_PATH,
        "Data/predictions/wordplay_interpretation_mistral_nemo_predictions.json"
    ),
    orient="index"
)

def clear_mistral_prediction(text_pred):
    
    text = text_pred[0]["generated_text"][-1]["content"]

    return text

df_mistral["mistral_nemo_preds"] = df_mistral["mistral_nemo_preds"].apply(clear_mistral_prediction)

In [12]:
df_mistral.sample(1)

,user_prompt,system_prompt,mistral_nemo_preds
2240,Заголовок новости: У коронавируса плохи аутоан...,\nПроанализируй заголовок новости в контексте ...,"В заголовке новости ""У коронавируса плохи ауто..."


In [13]:
df_gpt4o = pd.read_json(
    os.path.join(
        PROJECT_PATH,
        "Data/predictions/wordplay_interpretation_gpt_4o_predictions.json"
    ),
    orient="index"
)

In [14]:
df_gpt4o.sample(1)

,json_response,task_id,gpt4o_explain
780,{'id': 'batch_req_6796a6a9b0408190b7bf10c1ec0a...,780,в заголовке нет игры слов


In [15]:
dataset = pd.merge(dataset, df_gigachat_lite[["gigachat_lite_preds"]], left_index=True, right_index=True)
dataset = pd.merge(dataset, df_gigachat_max[["gigachat_max_pred"]], left_index=True, right_index=True)
dataset = pd.merge(dataset, df_yagpt[["yagpt_pred"]], left_index=True, right_index=True)
dataset = pd.merge(dataset, df_mistral[["mistral_nemo_preds"]], left_index=True, right_index=True)
dataset = pd.merge(dataset, df_gpt4o[["gpt4o_explain"]], left_index=True, right_index=True)

In [16]:
dataset.head(1)

,annotations,summary,is_word_play,date,article_url,headline,lead,gigachat_lite_preds,gigachat_max_pred,yagpt_pred,mistral_nemo_preds,gpt4o_explain
0,None,Жилищный нацпроект помимо количественных целей...,False,2021-12-16,https://www.kommersant.ru/doc/5130217,Комфортная среда,Городскую среду улучшают с учетом мнения жителей,В данном заголовке нет игры слов. Он просто ин...,в заголовке нет игры слов.,в заголовке нет игры слов. заголовок «комфортн...,"В заголовке новости ""Комфортная среда"" нет игр...",в заголовке нет игры слов


In [17]:
dataset["deepseak_explain"] = ["какой то текст для объяснения" for _ in range(len(dataset))]

In [18]:
dataset = dataset.dropna(subset=["annotations"])

In [19]:
def parse_one_annotation(annot):

    searcheable_elements = list()
    searcheable_links = list()

    if isinstance(annot["reference_string"], list):
        for i in range(len(annot["reference_string"])):
            searcheable_elements.append(
                annot["reference_string"][i]
            )
    elif isinstance(annot["reference_string"], str) and annot["reference_string"].strip() != "":
        searcheable_elements.append(
                annot["reference_string"]
        )

    if isinstance(annot["reference_url"], list):
        for i in range(len(annot["reference_url"])):
            searcheable_links.append(
                annot["reference_url"][i]
            )
    elif isinstance(annot["reference_url"], str) and annot["reference_url"].strip() != "":
        searcheable_links.append(
                annot["reference_url"]
        )

    return searcheable_elements, searcheable_links

In [20]:
def extract_searcheable_elements_and_links(annotations):

    searcheable_elements = list()
    searcheable_links = list()

    for annot in annotations:

        if isinstance(annot, dict):

            tmp_elements, tmp_links = parse_one_annotation(annot)
            searcheable_elements.extend(
                tmp_elements
            )
            searcheable_links.extend(
                tmp_links
            )

        elif isinstance(annot, list):

            for single_annot in annot:
                tmp_elements, tmp_links = parse_one_annotation(single_annot)
                searcheable_elements.extend(
                    tmp_elements
                )
                searcheable_links.extend(
                    tmp_links
                )
    
    return searcheable_elements, searcheable_links

In [21]:
dataset["annotations_len"] = dataset["annotations"].apply(lambda x: len(x))

In [22]:
dataset["searcheable_elements"] = dataset["annotations"].apply(lambda x: extract_searcheable_elements_and_links(x)[0])
dataset["searcheable_links"] = dataset["annotations"].apply(lambda x: extract_searcheable_elements_and_links(x)[1])

In [23]:
dataset["searcheable_elements_len"] = dataset["searcheable_elements"].apply(lambda x: len(x))
dataset["searcheable_links_len"] = dataset["searcheable_links"].apply(lambda x: len(x))

In [24]:
dataset = dataset.query("searcheable_elements_len >= 1 or searcheable_links_len >= 1")

In [25]:
def normalize_link(link):
    if "https://ru.wikipedia.org" in link:
        link = link.split("https://ru.wikipedia.org/wiki/")[1]
    elif "https://ru.wiktionary.org" in link:
        link = link.split("https://ru.wiktionary.org/wiki/")[1]
    
    if "#" in link:
        position = link.find("#")
        link = link[:position]
    
    link = re.sub(r'\(.*?\)', '', link).strip()

    link = re.sub(r'[^a-zA-Zа-яА-ЯёЁ]', ' ', link)
    link = re.sub(r'\s+', ' ', link).strip()

    return link.lower()

In [26]:
def tokenize(text: str):
    return " ".join(re.findall(r'\b\w+\b', text.lower()))

def lemmatize(tokens: str):
    tokens = tokens.split(" ")
    morph = pymorphy2.MorphAnalyzer()
    return " ".join([morph.parse(token)[0].normal_form for token in tokens])

def normalize_text(text):
    tokens = tokenize(text)
    lemmas = lemmatize(tokens)
    return " ".join(lemmas)

In [27]:
dataset["searcheable_elements"] = dataset["searcheable_elements"].apply(lambda x: [str.lower(t) for t in x])
dataset["searcheable_elements_tokenized"] = dataset["searcheable_elements"].apply(lambda x: [tokenize(t) for t in x])
dataset["searcheable_elements_normalized"] = dataset["searcheable_elements_tokenized"].apply(lambda x: [lemmatize(t) for t in x])

In [28]:
dataset["searcheable_links"] = dataset["searcheable_links"].apply(lambda x: [normalize_link(t) for t in x])
dataset["searcheable_links"] = dataset["searcheable_links"].apply(lambda x: [str.lower(t) for t in x])
dataset["searcheable_links_tokenized"] = dataset["searcheable_links"].apply(lambda x: [tokenize(t) for t in x])
dataset["searcheable_links_normalized"] = dataset["searcheable_links_tokenized"].apply(lambda x: [lemmatize(t) for t in x])

In [29]:
dataset["searcheable_all"] = dataset["searcheable_elements"] + dataset["searcheable_elements_tokenized"] + dataset["searcheable_elements_normalized"]

In [30]:
dataset.query(
    "article_url == 'https://www.kommersant.ru/doc/6209906'"
)

,annotations,summary,is_word_play,date,article_url,headline,lead,gigachat_lite_preds,gigachat_max_pred,yagpt_pred,...,annotations_len,searcheable_elements,searcheable_links,searcheable_elements_len,searcheable_links_len,searcheable_elements_tokenized,searcheable_elements_normalized,searcheable_links_tokenized,searcheable_links_normalized,searcheable_all
2014,"[{'headline_substring': 'Дели', 'start_index':...",Саммит лидеров G20 завершился подписанием итог...,True,2023-09-10,https://www.kommersant.ru/doc/6209906,На самом Дели,Что участники саммита G20 предпочли украинской...,"В заголовке новости ""На самом Дели"" использует...","что-то в вашем вопросе меня смущает. может, по...",в интернете есть много сайтов с информацией на...,...,2,"[деле, на са]",[],2,0,"[деле, на са]","[дело, на са]",[],[],"[деле, на са, деле, на са, дело, на са]"


In [31]:
def do_search(searcheable_elements, all_texts):
    
    results = list()

    assert isinstance(all_texts, list)
    assert len(all_texts) == 3

    for elem in searcheable_elements:
        for text in all_texts:
            results.append(
                str.find(text, elem)
            )
    return results

In [32]:
EXPLIAN_COLUMNS = ["deepseak_explain", "gigachat_lite_preds", "gigachat_max_pred", "yagpt_pred", "mistral_nemo_preds", "gpt4o_explain"]

for explain_column in EXPLIAN_COLUMNS:
    dataset[f"{explain_column}"] = dataset[f"{explain_column}"].apply(lambda x: str.lower(x))
    dataset[f"{explain_column}_tokenized"] = dataset[f"{explain_column}"].apply(lambda x: tokenize(x))
    dataset[f"{explain_column}_normalized"] = dataset[f"{explain_column}_tokenized"].apply(lambda x: lemmatize(x))
    dataset[f"{explain_column}"] = dataset[f"{explain_column}"].apply(lambda x: [x])
    dataset[f"{explain_column}_tokenized"] = dataset[f"{explain_column}_tokenized"].apply(lambda x: [x])
    dataset[f"{explain_column}_normalized"] = dataset[f"{explain_column}_normalized"].apply(lambda x: [x])
    dataset[f"{explain_column}_all"] = dataset[f"{explain_column}"] + dataset[f"{explain_column}_tokenized"] + dataset[f"{explain_column}_normalized"]

    dataset[f"{explain_column}_search_results"] = dataset.apply(lambda row: do_search(row["searcheable_all"], row[f"{explain_column}_all"]), axis=1)
    dataset[f"{explain_column}_search_results"] = dataset[f"{explain_column}_search_results"].apply(lambda x: max(x))

    dataset[f"{explain_column}_search_results"] = dataset[f"{explain_column}_search_results"].apply(lambda x: True if x >= 0 else False)




In [33]:
dataset.sample(2)

,annotations,summary,is_word_play,date,article_url,headline,lead,gigachat_lite_preds,gigachat_max_pred,yagpt_pred,...,yagpt_pred_all,yagpt_pred_search_results,mistral_nemo_preds_tokenized,mistral_nemo_preds_normalized,mistral_nemo_preds_all,mistral_nemo_preds_search_results,gpt4o_explain_tokenized,gpt4o_explain_normalized,gpt4o_explain_all,gpt4o_explain_search_results
1754,[{'headline_substring': 'Все подается в сравне...,“Ъ” ознакомился с рекомендациями по работе с и...,True,2022-11-07,https://www.kommersant.ru/doc/5457819,Все подается в сравнении,Единороссам посоветовали найти аналогии между ...,[в данном заголовке присутствует игра слов. \n...,"[не люблю менять тему разговора, но вот сейчас...",[в интернете есть много сайтов с информацией н...,...,[в интернете есть много сайтов с информацией н...,False,[в заголовке новости все подается в сравнении ...,[в заголовок новость всё подаваться в сравнени...,"[в заголовке новости ""все подается в сравнении...",False,[в заголовке нет игры слов],[в заголовок нет игра слово],"[в заголовке нет игры слов, в заголовке нет иг...",False
2142,[{'headline_substring': 'Заражен и особо конта...,Московская мэрия потратит 175 млн руб. на моде...,True,2021-01-10,https://www.kommersant.ru/doc/5009292,Заражен и особо контактен,Московская мэрия будет тщательнее следить за б...,[в данном заголовке нет игры слов.<|message_se...,[в заголовке нет игры слов.],[в заголовке нет игры слов.],...,"[в заголовке нет игры слов., в заголовке нет и...",False,[в заголовке новости есть игра слов слово конт...,[в заголовок новость есть игра слово слово кон...,"[в заголовке новости есть игра слов. слово ""ко...",False,[в заголовке нет игры слов],[в заголовок нет игра слово],"[в заголовке нет игры слов, в заголовке нет иг...",False


In [35]:
dataset[[f"{t}_search_results" for t in EXPLIAN_COLUMNS]].sum() / len(dataset)

deepseak_explain_search_results       0.000968
gigachat_lite_preds_search_results    0.167473
gigachat_max_pred_search_results      0.140368
yagpt_pred_search_results             0.167473
mistral_nemo_preds_search_results     0.232333
gpt4o_explain_search_results          0.238141
dtype: float64